In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf

In [ ]:
import tensorflow_datasets as tfds

Loading the dataset from tensorflow datasets

In [ ]:
#split argument will shuffle the dataset for splitting so that train and test data contains both the classes
(train_data,val_data),info=tfds.load('cats_vs_dogs',split=['train[:80%]','train[80%:]'],with_info=True,as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cats_vs_dogs/incomplete.6Z0NWO_4.0.1/cats_vs_dogs-train.tfrecord*...:   0%…

Dataset cats_vs_dogs downloaded and prepared to /root/tensorflow_datasets/cats_vs_dogs/4.0.1. Subsequent calls will reuse this data.


In [ ]:
info

tfds.core.DatasetInfo(
    name='cats_vs_dogs',
    full_name='cats_vs_dogs/4.0.1',
    description="""
    A large set of images of cats and dogs. There are 1738 corrupted images that are dropped.
    """,
    homepage='https://www.microsoft.com/en-us/download/details.aspx?id=54765',
    data_dir='/root/tensorflow_datasets/cats_vs_dogs/4.0.1',
    file_format=tfrecord,
    download_size=786.67 MiB,
    dataset_size=1.04 GiB,
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=uint8),
        'image/filename': Text(shape=(), dtype=string),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=2),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    nondeterministic_order=False,
    splits={
        'train': <SplitInfo num_examples=23262, num_shards=16>,
    },
    citation="""@Inproceedings (Conference){asirra-a-captcha-that-exploits-interest-aligned-manual-image-categorization,
    author = {Elson, Jeremy and Douceur, Jo

Preprocessing the dataset

In [ ]:
def resize_images(image,label):
  image=tf.image.resize(image,(160,160))
  image=image/255.0
  return image,label


In [ ]:
batch_size=32

In [ ]:
train = (train_data
         .map(resize_images)
         .shuffle(1000)
         .batch(batch_size)
         .prefetch(tf.data.AUTOTUNE))

val = (val_data
       .map(resize_images)
       .batch(batch_size
              )
       .prefetch(tf.data.AUTOTUNE))

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    # Input layer (resized to 160x160 with 3 color channels)
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Dropout(0.5),  # Helps prevent overfitting

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification: cat vs dog
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
EPOCHS = 10

history = model.fit(
    train,
    validation_data=val,
    epochs=EPOCHS
)


Epoch 1/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 45s 63ms/step - accuracy: 0.5386 - loss: 0.6868 - val_accuracy: 0.6950 - val_loss: 0.5853
Epoch 2/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 30s 49ms/step - accuracy: 0.6795 - loss: 0.6004 - val_accuracy: 0.7678 - val_loss: 0.4947
Epoch 3/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 31s 50ms/step - accuracy: 0.7549 - loss: 0.5005 - val_accuracy: 0.7788 - val_loss: 0.4616
Epoch 4/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 46s 57ms/step - accuracy: 0.8115 - loss: 0.4110 - val_accuracy: 0.8375 - val_loss: 0.3739
Epoch 5/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 35s 58ms/step - accuracy: 0.8402 - loss: 0.3602 - val_accuracy: 0.8669 - val_loss: 0.3197
Epoch 6/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 35s 48ms/step - accuracy: 0.8663 - loss: 0.3160 - val_accuracy: 0.8693 - val_loss: 0.3069
Epoch 7/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 29s 49ms/step - accuracy: 0.8892 - loss: 0.2667 - val_accuracy: 0.8820 - val_loss: 0.2904
Epoch 8/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 31s 51ms/step - accuracy: 0.9030 - loss: 0.2376 - 

In [ ]:
print("Final Training Accuracy:", history.history['accuracy'][-1])
print("Final Validation Accuracy:", history.history['val_accuracy'][-1])

print("Final Training Loss:", history.history['loss'][-1])
print("Final Validation Loss:", history.history['val_loss'][-1])


Final Training Accuracy: 0.9323481917381287
Final Validation Accuracy: 0.8942390084266663
Final Training Loss: 0.16893836855888367
Final Validation Loss: 0.259405255317688


Model

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    # Input layer (resized to 160x160 with 3 color channels)
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Dropout(0.5),  # Helps prevent overfitting

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(128,activation='relu'),
    layers.Dense(64,activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification: cat vs dog
])


In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
EPOCHS = 10

history = model.fit(
    train,
    validation_data=val,
    epochs=EPOCHS
)

Epoch 1/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 38s 54ms/step - accuracy: 0.5595 - loss: 0.6727 - val_accuracy: 0.6829 - val_loss: 0.5911
Epoch 2/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 35s 58ms/step - accuracy: 0.7097 - loss: 0.5653 - val_accuracy: 0.7440 - val_loss: 0.4909
Epoch 3/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 35s 58ms/step - accuracy: 0.7765 - loss: 0.4701 - val_accuracy: 0.7846 - val_loss: 0.4545
Epoch 4/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 30s 49ms/step - accuracy: 0.8144 - loss: 0.4094 - val_accuracy: 0.8222 - val_loss: 0.3935
Epoch 5/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 41s 50ms/step - accuracy: 0.8401 - loss: 0.3628 - val_accuracy: 0.8491 - val_loss: 0.3459
Epoch 6/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 40s 48ms/step - accuracy: 0.8628 - loss: 0.3213 - val_accuracy: 0.8592 - val_loss: 0.3237
Epoch 7/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 36s 60ms/step - accuracy: 0.8801 - loss: 0.2814 - val_accuracy: 0.8485 - val_loss: 0.3899
Epoch 8/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 35s 49ms/step - accuracy: 0.8974 - loss: 0.2433 - 

In [ ]:
print("Final Training Accuracy:", history.history['accuracy'][-1])
print("Final Validation Accuracy:", history.history['val_accuracy'][-1])

print("Final Training Loss:", history.history['loss'][-1])
print("Final Validation Loss:", history.history['val_loss'][-1])


Final Training Accuracy: 0.929016649723053
Final Validation Accuracy: 0.8658641576766968
Final Training Loss: 0.17560195922851562
Final Validation Loss: 0.4290439486503601


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    # Input layer (resized to 160x160 with 3 color channels)
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Dropout(0.5),  # Helps prevent overfitting

    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(64,activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification: cat vs dog
])


In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
EPOCHS = 10

history = model.fit(
    train,
    validation_data=val,
    epochs=EPOCHS
)

Epoch 1/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 37s 54ms/step - accuracy: 0.5617 - loss: 0.6723 - val_accuracy: 0.6713 - val_loss: 0.5949
Epoch 2/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 30s 49ms/step - accuracy: 0.7183 - loss: 0.5516 - val_accuracy: 0.7577 - val_loss: 0.5110
Epoch 3/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 35s 57ms/step - accuracy: 0.7795 - loss: 0.4670 - val_accuracy: 0.7973 - val_loss: 0.4560
Epoch 4/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 35s 48ms/step - accuracy: 0.8183 - loss: 0.4002 - val_accuracy: 0.8485 - val_loss: 0.3481
Epoch 5/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 30s 49ms/step - accuracy: 0.8522 - loss: 0.3380 - val_accuracy: 0.8665 - val_loss: 0.3052
Epoch 6/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 34s 56ms/step - accuracy: 0.8772 - loss: 0.2933 - val_accuracy: 0.8708 - val_loss: 0.3057
Epoch 7/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 37s 51ms/step - accuracy: 0.8933 - loss: 0.2530 - val_accuracy: 0.8654 - val_loss: 0.3138
Epoch 8/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 36s 60ms/step - accuracy: 0.9104 - loss: 0.2140 - 

In [ ]:
print("Final Training Accuracy:", history.history['accuracy'][-1])
print("Final Validation Accuracy:", history.history['val_accuracy'][-1])

print("Final Training Loss:", history.history['loss'][-1])
print("Final Validation Loss:", history.history['val_loss'][-1])


Final Training Accuracy: 0.9339064955711365
Final Validation Accuracy: 0.8813413381576538
Final Training Loss: 0.16028989851474762
Final Validation Loss: 0.2941605746746063


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    # Input layer (resized to 160x160 with 3 color channels)
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Dropout(0.5),  # Helps prevent overfitting

    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification: cat vs dog
])


In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
EPOCHS = 10

history = model.fit(
    train,
    validation_data=val,
    epochs=EPOCHS
)

Epoch 1/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 41s 62ms/step - accuracy: 0.5332 - loss: 0.6879 - val_accuracy: 0.6754 - val_loss: 0.6094
Epoch 2/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 29s 49ms/step - accuracy: 0.6929 - loss: 0.5766 - val_accuracy: 0.7745 - val_loss: 0.4864
Epoch 3/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 34s 56ms/step - accuracy: 0.7800 - loss: 0.4645 - val_accuracy: 0.8205 - val_loss: 0.4001
Epoch 4/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 34s 57ms/step - accuracy: 0.8187 - loss: 0.4009 - val_accuracy: 0.8446 - val_loss: 0.3567
Epoch 5/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 35s 47ms/step - accuracy: 0.8510 - loss: 0.3419 - val_accuracy: 0.8603 - val_loss: 0.3186
Epoch 6/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 41s 48ms/step - accuracy: 0.8723 - loss: 0.2975 - val_accuracy: 0.8665 - val_loss: 0.3191
Epoch 7/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 31s 50ms/step - accuracy: 0.8909 - loss: 0.2588 - val_accuracy: 0.8850 - val_loss: 0.2857
Epoch 8/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 30s 49ms/step - accuracy: 0.9101 - loss: 0.2213 - 

In [ ]:
print("Final Training Accuracy:", history.history['accuracy'][-1])
print("Final Validation Accuracy:", history.history['val_accuracy'][-1])

print("Final Training Loss:", history.history['loss'][-1])
print("Final Validation Loss:", history.history['val_loss'][-1])


Final Training Accuracy: 0.9329929947853088
Final Validation Accuracy: 0.8888649940490723
Final Training Loss: 0.16471828520298004
Final Validation Loss: 0.2852931022644043


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    # Input layer (resized to 160x160 with 3 color channels)
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Dropout(0.5),  # Helps prevent overfitting

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification: cat vs dog
])


In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
EPOCHS = 10

history = model.fit(
    train,
    validation_data=val,
    epochs=EPOCHS
)

Epoch 1/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 48s 67ms/step - accuracy: 0.5486 - loss: 0.6808 - val_accuracy: 0.7070 - val_loss: 0.5707
Epoch 2/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 72s 60ms/step - accuracy: 0.7178 - loss: 0.5534 - val_accuracy: 0.7715 - val_loss: 0.4780
Epoch 3/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 31s 51ms/step - accuracy: 0.7897 - loss: 0.4489 - val_accuracy: 0.8347 - val_loss: 0.3807
Epoch 4/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 31s 50ms/step - accuracy: 0.8332 - loss: 0.3785 - val_accuracy: 0.8422 - val_loss: 0.3457
Epoch 5/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 30s 50ms/step - accuracy: 0.8581 - loss: 0.3267 - val_accuracy: 0.8581 - val_loss: 0.3319
Epoch 6/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 36s 59ms/step - accuracy: 0.8788 - loss: 0.2858 - val_accuracy: 0.8691 - val_loss: 0.3150
Epoch 7/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 32s 52ms/step - accuracy: 0.8900 - loss: 0.2565 - val_accuracy: 0.8925 - val_loss: 0.2811
Epoch 8/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 31s 51ms/step - accuracy: 0.9023 - loss: 0.2285 - 

In [ ]:
print("Final Training Accuracy:", history.history['accuracy'][-1])
print("Final Validation Accuracy:", history.history['val_accuracy'][-1])

print("Final Training Loss:", history.history['loss'][-1])
print("Final Validation Loss:", history.history['val_loss'][-1])


Final Training Accuracy: 0.9380440711975098
Final Validation Accuracy: 0.90025794506073
Final Training Loss: 0.15533341467380524
Final Validation Loss: 0.2757143974304199


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    # Input layer (resized to 160x160 with 3 color channels)
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Dropout(0.5),  # Helps prevent overfitting

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification: cat vs dog
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
EPOCHS = 10

history1 = model.fit(
    train,
    validation_data=val,
    epochs=EPOCHS
)

Epoch 1/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 46s 69ms/step - accuracy: 0.5262 - loss: 0.6913 - val_accuracy: 0.5849 - val_loss: 0.6633
Epoch 2/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 30s 49ms/step - accuracy: 0.6306 - loss: 0.6387 - val_accuracy: 0.7208 - val_loss: 0.5425
Epoch 3/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 36s 60ms/step - accuracy: 0.7409 - loss: 0.5217 - val_accuracy: 0.7958 - val_loss: 0.4312
Epoch 4/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 31s 51ms/step - accuracy: 0.7840 - loss: 0.4553 - val_accuracy: 0.8156 - val_loss: 0.3963
Epoch 5/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 32s 52ms/step - accuracy: 0.8195 - loss: 0.3989 - val_accuracy: 0.8229 - val_loss: 0.3848
Epoch 6/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 31s 51ms/step - accuracy: 0.8472 - loss: 0.3456 - val_accuracy: 0.8418 - val_loss: 0.3525
Epoch 7/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 36s 59ms/step - accuracy: 0.8665 - loss: 0.3076 - val_accuracy: 0.8669 - val_loss: 0.3052
Epoch 8/10
582/582 ━━━━━━━━━━━━━━━━━━━━ 36s 51ms/step - accuracy: 0.8855 - loss: 0.2729 - 

In [ ]:
print("Final Training Accuracy:", history1.history['accuracy'][-1])
print("Final Validation Accuracy:", history1.history['val_accuracy'][-1])

print("Final Training Loss:", history1.history['loss'][-1])
print("Final Validation Loss:", history1.history['val_loss'][-1])


Final Training Accuracy: 0.917141318321228
Final Validation Accuracy: 0.8901547789573669
Final Training Loss: 0.20119018852710724
Final Validation Loss: 0.2896167039871216


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

model = models.Sequential([
    # Input layer (resized to 160x160 with 3 color channels)
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(160, 160, 3)),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),

    layers.Dropout(0.5),  # Helps prevent overfitting

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification: cat vs dog
])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [ ]:
EPOCHS = 15

history4 = model.fit(
    train,
    validation_data=val,
    epochs=EPOCHS
)

Epoch 1/15
582/582 ━━━━━━━━━━━━━━━━━━━━ 43s 59ms/step - accuracy: 0.5641 - loss: 0.6802 - val_accuracy: 0.7242 - val_loss: 0.5557
Epoch 2/15
582/582 ━━━━━━━━━━━━━━━━━━━━ 30s 50ms/step - accuracy: 0.7182 - loss: 0.5553 - val_accuracy: 0.7616 - val_loss: 0.4908
Epoch 3/15
582/582 ━━━━━━━━━━━━━━━━━━━━ 30s 50ms/step - accuracy: 0.7845 - loss: 0.4586 - val_accuracy: 0.8248 - val_loss: 0.3967
Epoch 4/15
582/582 ━━━━━━━━━━━━━━━━━━━━ 42s 51ms/step - accuracy: 0.8248 - loss: 0.3919 - val_accuracy: 0.8506 - val_loss: 0.3548
Epoch 5/15
582/582 ━━━━━━━━━━━━━━━━━━━━ 30s 50ms/step - accuracy: 0.8498 - loss: 0.3431 - val_accuracy: 0.8315 - val_loss: 0.3867
Epoch 6/15
582/582 ━━━━━━━━━━━━━━━━━━━━ 47s 59ms/step - accuracy: 0.8716 - loss: 0.2941 - val_accuracy: 0.8702 - val_loss: 0.3035
Epoch 7/15
582/582 ━━━━━━━━━━━━━━━━━━━━ 36s 59ms/step - accuracy: 0.8871 - loss: 0.2603 - val_accuracy: 0.8674 - val_loss: 0.3214
Epoch 8/15
582/582 ━━━━━━━━━━━━━━━━━━━━ 30s 50ms/step - accuracy: 0.9042 - loss: 0.2267 - 

In [ ]:
print("Final Training Accuracy:", history4.history['accuracy'][-1])
print("Final Validation Accuracy:", history4.history['val_accuracy'][-1])

print("Final Training Loss:", history4.history['loss'][-1])
print("Final Validation Loss:", history4.history['val_loss'][-1])


Final Training Accuracy: 0.9684040546417236
Final Validation Accuracy: 0.9028375148773193
Final Training Loss: 0.08728204667568207
Final Validation Loss: 0.3142739236354828


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

def predict_image(model, image_path):
    # Load and preprocess image
    img = load_img(image_path, target_size=(160, 160))  # Resize to match model input
    img_array = img_to_array(img) / 255.0  # Normalize pixel values
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

    # Make prediction
    prediction = model.predict(img_array)[0][0]

    # Interpret result
    print(f"Prediction score: {prediction:.4f}")
    if prediction > 0.5:
        print("Predicted: Dog 🐶")
    else:
        print("Predicted: Cat 🐱")


In [ ]:
predict_image(model, '/content/cat1.jfif')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 857ms/step
Prediction score: 0.0006
Predicted: Cat 🐱


In [ ]:
predict_image(model, '/content/cat2.jpg')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Prediction score: 0.1305
Predicted: Cat 🐱


In [ ]:
predict_image(model, '/content/Dog1.webp')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Prediction score: 0.9995
Predicted: Dog 🐶


In [ ]:
predict_image(model, '/content/Dog2.webp')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
Prediction score: 0.9999
Predicted: Dog 🐶


In [ ]:
model.save("cats_vs_dogs_cnn.h5")